In [1]:
include("LiPoSID.jl")
using LinearAlgebra

depolarizing chanel

$\sqrt{1-\frac{3p}{4}}\begin{bmatrix} 
1 & 0 \\ 0 & 1
\end{bmatrix}, \quad \sqrt{\frac{p}{4}}\begin{bmatrix}
0 & 1 \\ 1 & 0 \end{bmatrix},
 \quad \sqrt{\frac{p}{4}}\begin{bmatrix}
0 & -i \\ i & 0 \end{bmatrix},
 \quad \sqrt{\frac{p}{4}}\begin{bmatrix}
1 & 0 \\ 0 & -1 \end{bmatrix}$

In [2]:
σ = [ [0 1; 1 0], [0 -im; im 0], [1 0; 0 -1], [1 0; 0 1] ]
σˣ = σ[1]; σʸ = σ[2]; σᶻ = σ[3]; σᴵ = σ[4]; 

p = 0.123456789
E = []
push!(E, sqrt(1-3p/4)σᴵ)
        
push!(E, sqrt(p/4)σˣ)

push!(E, sqrt(p/4)σʸ)
     
push!(E, sqrt(p/4)σᶻ)

sum(e'e for e in E)

2×2 Matrix{ComplexF64}:
 1.0+0.0im  0.0+0.0im
 0.0+0.0im  1.0+0.0im

In [3]:
ρᵍ₀ = [1. 0]' * [1 0]
ρᵉ₀ = [0. 1]' * [0 1]
ρˣ₀ = [1 1]' * [1 1]/2
ρʸ₀ = [1 -im]' * [1 -im]/2

2×2 Matrix{ComplexF64}:
 0.5+0.0im  0.0-0.5im
 0.0+0.5im  0.5+0.0im

In [4]:
1 - 3p/4 

0.90740740825

In [5]:
t_steps = 2
ρᵍ = LiPoSID.timeevolution_kraus(t_steps, ρᵍ₀, E)
ρᵉ = LiPoSID.timeevolution_kraus(t_steps, ρᵉ₀, E)
ρˣ = LiPoSID.timeevolution_kraus(t_steps, ρˣ₀, E)
ρʸ = LiPoSID.timeevolution_kraus(t_steps, ρʸ₀, E)

A = LiPoSID.direct_DMD_01XY_b4_A([ρᵉ, ρᵍ, ρˣ, ρʸ])

A
# A[abs.(A) .< 1e-6] .= 0
A

4×4 Matrix{Float64}:
  0.876543     1.60037e-16   9.04583e-17   5.19703e-17
 -2.70481e-16  0.876543      2.16456e-17   6.03413e-17
  7.92233e-16  8.61451e-16   0.876543     -5.20845e-16
  5.79601e-16  2.22045e-16  -9.41515e-17   1.0

In [6]:
LiPoSID.timeevolution_kraus(2, ρˣ₀, E)[end]

2×2 Matrix{Float64}:
 0.5       0.438272
 0.438272  0.5

In [7]:
𝓔(ρˣ₀, A)

UndefVarError: UndefVarError: `𝓔` not defined

$\rho_1' = \mathcal{E}(|0\rangle \langle 0 |)$

$\rho_4' = \mathcal{E}(|1\rangle \langle 1 |)$

$\rho_2' = \mathcal{E}(|+\rangle \langle + |) -i  \mathcal{E}(|-\rangle \langle-|) - (1\textbf{+}i) (\rho_1' + \rho_4')/2, \quad$ - it is $(1\textbf{-}i)$ in Nielsen&Chuang which is WRONG!

$\rho_3' = \mathcal{E}(|+\rangle \langle + |) +i  \mathcal{E}(|-\rangle \langle-|) - (1+i) (\rho_1' + \rho_4')/2$

In [41]:
function bloch4(ρ)

    b = convert.(Float64, [ ρ[1,2] + ρ[2,1],
                           (ρ[1,2] - ρ[2,1])*im,
                            ρ[1,1] - ρ[2,2],
                            ρ[1,1] + ρ[2,2]       ])

end

function dm_b4(b) 

    ρ = [ 1+b[3]         b[2]-im*b[3]
          b[2]+im*b[3]   1-b[3]       ]/2

end

function 𝓔(ρ₀, A)
    ρᶠ = dm_b4(A * bloch4(ρ₀))
    return ρᶠ
end

function QPT(A)

    ρᵍ₀ = [1. 0]' * [1 0]
    ρᵉ₀ = [0. 1]' * [0 1]
    ρˣ₀ = [1 1]' * [1 1]/2
    ρʸ₀ = [1 -im]' * [1 -im]/2

    ρ₁ = 𝓔(ρᵍ₀, A)
    ρ₄ = 𝓔(ρᵉ₀, A)
    ρ₃= 𝓔(ρˣ₀, A) - im*𝓔(ρʸ₀, A) - (1-im)*(ρ₁+ρ₄)/2 # with "-" sign χ can be diagonalized ???
    ρ₂= 𝓔(ρˣ₀, A) + im*𝓔(ρʸ₀, A) - (1+im)*(ρ₁+ρ₄)/2 

    #ρ₂ = 𝓔(ρˣ₀, Ac) - im*𝓔(ρʸ₀, Ac) - (1-im)*(ρ₁+ρ₄)/2 # with "-" sign χ can be diagonalized ???
    #ρ₃ = 𝓔(ρˣ₀, Ac) + im*𝓔(ρʸ₀, Ac) - (1+im)*(ρ₁+ρ₄)/2 

    σₓ = [ 0  1
           1  0 ]  # X gate

    Λ = [ I  σₓ
          σₓ  -I ] / 2

    #Ρ = [ρ₁ ρ₃ 
    #     ρ₂ ρ₄] 

    Ρ = [ρ₁ ρ₂
         ρ₃ ρ₄] 
     
    χ = Λ*Ρ*Λ

    return χ 
end

χ = QPT(A)
d, U = eigen(χ)

U * diagm(d) * U' ≈ χ

true

In [8]:
ishermitian(χ)

UndefVarError: UndefVarError: `χ` not defined

In [9]:
d

UndefVarError: UndefVarError: `d` not defined

In [30]:
d[d .< 0] .= 0
d

4-element Vector{Float64}:
 0.0
 0.0
 0.5599048242505645
 0.5599048242505651

In [10]:
ishermitian(χ)

true

In [11]:
function operator_sum(χ)

    @assert ishermitian(χ)

    E₀ = [1.  0
          0   1 ]
    E₁ = [ 0.  1
           1   0 ]  #  σₓ  or X gate
    E₂ = -im * [ 0  -im
                im   0  ]
    E₃ = [ 1  0
           0 -1 ]
    Ẽ = [E₀, E₁, E₂, E₃] 

    d, U = eigen(χ)
    #@assert U * diagm(d) * U' ≈ χ
    E = []
    for i in 1:size(U)[1]
        Eᵢ = sqrt(Complex(d[i])) * sum(U[j,i] * Ẽ[j] for j in 1:size(U)[2])
        Eᵢ[abs.(Eᵢ) .< 1e-5] .= 0
        push!(E, Eᵢ)
    end
    return E
end

Eᵗᵒᵐ = operator_sum(χ)

sum(e'e for e in Eᵗᵒᵐ)

2×2 Matrix{ComplexF64}:
     1.23962+0.0im  3.23191e-17+0.0im
 3.23191e-17+0.0im      1.23962+0.0im

In [12]:
Eᵗᵒᵐ[1]

2×2 Matrix{ComplexF64}:
   -0.0936635-7.87498e-17im  2.1104e-16+0.226124im
 -5.56053e-16+0.226124im     -0.0936635-7.87498e-17im

In [13]:
E[1]

2×2 Matrix{ComplexF64}:
 0.952579+0.0im       0.0+0.0im
      0.0+0.0im  0.952579+0.0im

In [14]:
Eᵗᵒᵐ[2]

2×2 Matrix{ComplexF64}:
 2.69088e-16-0.0936635im       0.226124-2.72086e-16im
   -0.226124-2.72086e-16im  2.69088e-16+0.0936635im

In [15]:
E[2]

2×2 Matrix{ComplexF64}:
      0.0+0.0im  0.175682+0.0im
 0.175682+0.0im       0.0+0.0im

In [16]:
Eᵗᵒᵐ[3]

2×2 Matrix{ComplexF64}:
 -1.61793e-15+0.691309im         -0.28635-1.12251e-15im
     -0.28635-1.66302e-16im  -1.61793e-15+0.691309im

In [17]:
E[3]

2×2 Matrix{ComplexF64}:
 0.0+0.0im       0.0-0.175682im
 0.0+0.175682im  0.0+0.0im

In [18]:
Eᵗᵒᵐ[4]

2×2 Matrix{ComplexF64}:
     0.691309-3.40757e-16im  -3.44554e-16-0.28635im
 -3.44554e-16+0.28635im         -0.691309-3.40757e-16im

In [19]:
E[4]

2×2 Matrix{ComplexF64}:
 0.175682+0.0im        0.0+0.0im
      0.0+0.0im  -0.175682+0.0im